In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [2]:
import tensorflow_datasets as tfds
#tfds.list_builders()
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)





Shuffling and writing examples to C:\Users\ARGTLP002\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete81SIR1\imdb_reviews-train.tfrecord
Shuffling and writing examples to C:\Users\ARGTLP002\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete81SIR1\imdb_reviews-test.tfrecord
Shuffling and writing examples to C:\Users\ARGTLP002\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incomplete81SIR1\imdb_reviews-unsupervised.tfrecord
Dataset imdb_reviews downloaded and prepared to C:\Users\ARGTLP002\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [5]:
import numpy as np

training_sentences  = []
training_labels     = []
training_labels_temp= []

testing_sentences   = []
testing_labels      = []
testing_labels_temp = []

train_data, test_data = imdb['train'], imdb['test']

for sentence, label in train_data:
    training_sentences.append(str(sentence.numpy()))
    training_labels_temp.append(label.numpy())

training_labels = np.array(training_labels_temp)


for sentence, label in test_data:
    testing_sentences.append(str(sentence.numpy()))
    testing_labels_temp.append(label.numpy())

testing_labels = np.array(testing_labels_temp)


In [10]:
vocab_size = 10000
embedding_dim = 16
max_len =120
trunc_type = 'post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(training_sentences)
train_padded = pad_sequences(train_sequences, maxlen = max_len, truncating = trunc_type)

test_sequences = tokenizer.texts_to_sequences(testing_sentences)
test_padded = pad_sequences(test_sequences, maxlen = max_len, truncating = trunc_type)



In [15]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(train_padded[2]))
print(training_sentences[2])



<OOV> photographs the <OOV> rocky mountains in a superb fashion and jimmy stewart and walter brennan give enjoyable performances as they always seem to do br br but come on hollywood a <OOV> telling the people of dawson city <OOV> to <OOV> themselves a <OOV> yes a <OOV> and to <OOV> the law themselves then <OOV> battling it out on the streets for control of the town br br nothing even remotely resembling that happened on the canadian side of the border during the <OOV> gold rush mr mann and company appear to have mistaken dawson city for <OOV> the canadian north for the american wild west br br canadian viewers be prepared for a <OOV> madness type of enjoyable
b'Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Mountie telling the people of Dawson City, Yukon to elect themselves a marshal (yes a marshal!) and to enforce the law themselves, th

In [18]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(6, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [19]:
num_epochs = 10

model.fit(train_padded, training_labels, epochs=num_epochs, validation_data=(test_padded, testing_labels))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.4919 - accuracy: 0.7498 - val_loss: 0.3837 - val_accuracy: 0.8281

In [20]:
layer = model.layers[0]
weights = layer.get_weights()[0]
print(weights.shape)

(10000, 16)


In [22]:
import io

out_v = io.open('C:/VIN_PYCHARM/ONE/Data/vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('C:/VIN_PYCHARM/ONE/Data/meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()